In [1]:
import copy
import json
import os
import sys
import time

In [2]:
from icecream import ic
import numpy as np
import pandas as pd
from pprint import pprint

import torch
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import densenet

from tqdm import tqdm

In [3]:
os.chdir('../')
sys.path.append('../dataset/')
import coco_data_prep
import coco_api_helper
import config_dataset

sys.path.append('../models/')
import densenet_inspired as di

loading annotations into memory...
Done (t=12.86s)
creating index...
index created!
loading annotations into memory...
Done (t=5.49s)
creating index...
index created!
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [4]:
%load_ext autoreload

In [5]:
%autoreload

### Create the Cateogries Label file

In [6]:
# cat_ids = [annot['id'] for annot in train_annot['categories']]
# cat_names = [annot['name'] for annot in train_annot['categories']]

# cat_table = pd.DataFrame(pd.Series(cat_ids, cat_names))
# cat_table.reset_index(drop=False, inplace=True)
# cat_table.rename(columns={
#     'index': 0,
#     0: 1
# }, inplace=True)
# cat_table = cat_table[[1, 0]]
# cat_table.to_csv('../dataset/coco_labels.txt', header=None)

### Helper Functions

In [7]:
def calc_new_dim(orig_h_or_w: int, padding, kernel, stride) -> int:
    """
    :orig_h_or_w: original width or original height of the input image
    given the original width or height, kernel size, pad width, stride size
    calculate the new width or height
    """
    new_dim = (orig_h_or_w + (2 * padding) - kernel) // stride + 1
    return new_dim

### Global Variables

In [8]:
with open('../dataset/categories.json', 'r') as j:
    categories = json.load(j)    

# catid_to_catname_df = pd.DataFrame.from_records(pd.Series([c.values() for c in categories])).drop(0, axis=1)
# catid_to_catname_df.to_csv('../dataset/coco_labels.txt', header=None, sep=',', mode='a')

In [9]:
train_np_data_dir = '../data/numpy_imgs/train_subset/'
train_jpg_data_dir = '../data/raw/train/train2014/'
train_annot_filepath = '../data/raw/train/annotations/instances_train2014.json'

with open('../dataset/imgs_by_supercategory.json', 'r') as f:
    desired_categories = json.load(f)

In [10]:
val_np_data_dir = '../data/numpy_imgs/val_subset/'
val_jpg_data_dir = '../data/raw/val/val2014/'
val_annot_filepath = '../data/raw/train/annotations/instances_val2014.json'

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')

### Load Data : Train & Validation into DataLoader

In [12]:
train_ds = coco_data_prep.COCODataset('train',
                                      train_np_data_dir, 
                                      train_annot_filepath,
                                      coco_data_prep.COCOAnnotationTransform(),
                                      sample_ratio=0.05,
                                      device='cpu')

loading annotations into memory...
Done (t=13.49s)
creating index...
index created!


100%|███████████████████████████████████████████| 82783/82783 [00:48<00:00, 1720.90it/s]


In [13]:
train_dl = coco_data_prep.get_dataloader(train_ds, 
                                         batch_size=512, 
                                         device='cpu')

In [14]:
val_ds = coco_data_prep.COCODataset('val',
                                    val_np_data_dir, 
                                    val_annot_filepath,
                                    coco_data_prep.COCOAnnotationTransform(),
                                    sample_ratio = 0.05,
                                    device='cpu')

loading annotations into memory...
Done (t=4.25s)
creating index...
index created!


100%|███████████████████████████████████████████| 40504/40504 [00:27<00:00, 1495.47it/s]


In [15]:
val_dl = coco_data_prep.get_dataloader(val_ds, 
                                     batch_size=512, 
                                     device='cpu')

### Load Model
- see model in `models/densenet_inspired.py`

In [16]:
model = di.DensenetInspired()

/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/torch/nn/modules/lazy.py:175: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [17]:
model

DensenetInspired(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (mp1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU(inplace=True)
  (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (lin1): LazyLinear(in_features=0, out_features=1024, bias=True)
  (fc1): Linear(in_features=1024, out_features=80, bias=True)
)

#### Modeling Variables

In [18]:
optim_params= dict(
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-05,
    weight_decay=0.01,
    amsgrad=False,
)

optimizer = torch.optim.Adam(model.parameters(), **optim_params)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [19]:
criterion = nn.CrossEntropyLoss()

In [20]:
torch.cuda.empty_cache()

In [21]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        

In [23]:
best = 1000
best_cm = None
best_model = None


for epoch in range(25):
    iter_time = AverageMeter()
    losses = AverageMeter()
    running_loss = 0.0
    count = 0
    
    # train loop
    for idx, (batch) in enumerate(train_dl):
        start = time.time()
        
        image_batch, label_batch = [x[0] for x in batch], [x[1] for x in batch]
        image_batch = torch.stack(image_batch).to('cpu')
        label_batch = torch.LongTensor([l[0] for l in label_batch])

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward (model & loss), Backward, and Optimize (Update weights)
        out = model(image_batch)
        
        loss = criterion(out, label_batch)
        print(loss)

        loss.backward()
        optimizer.step()
        count += 1

        losses.update(loss, out.shape[0])
        running_loss += loss

        iter_time.update(time.time() - start)
        if idx % 10 == 0:
            print(('Epoch: [{0}][{1}/{2}]\t'
                   'Time {iter_time.val:.3f} ({iter_time.avg:.3f})\t'
                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t')
                   .format(epoch, idx, len(train_dl), iter_time=iter_time, loss=losses))
    
    print(f'Epoch-{epoch} lr: {optimizer.param_groups[0]["lr"]} running_loss: {running_loss/image_batch.shape[0]}')
    
    
    # validation loop
    iter_time = AverageMeter()
    losses = AverageMeter()

    num_class = 80
    cm =torch.zeros(num_class, num_class)
    # evaluation loop
    for idx, (batch) in enumerate(val_dl):
        start = time.time()

        image_batch, label_batch = [x[0] for x in batch], [x[1] for x in batch]
        image_batch = torch.stack(image_batch).to('cpu')
        label_batch = torch.LongTensor([l[0] for l in label_batch])

        model.eval()
        torch.no_grad()

        out = model(image_batch)
        loss = criterion(out, label_batch)
        losses.update(loss, out.shape[0])

        iter_time.update(time.time() - start)
        if idx % 10 == 0:
            print(('Epoch: [{0}][{1}/{2}]\t'
               'Time {iter_time.val:.3f} ({iter_time.avg:.3f})\t')
               .format(epoch, 
                       idx, 
                       len(val_dl), 
                       iter_time=iter_time, 
                       loss=losses, top1=0))
            
        if loss < best:
            best = loss
            best_model = copy.deepcopy(model)
            torch.save({
                'epoch': epoch,
                'model_state_dict': best_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss}, f'./checkpoints/densenet_inspired_epoch_{epoch}.pth')
            
    

tensor(4.3713, grad_fn=<NllLossBackward>)
Epoch: [0][0/9]	Time 5.955 (5.955)	Loss 4.3713 (4.3713)	
tensor(213.0275, grad_fn=<NllLossBackward>)
tensor(134.5100, grad_fn=<NllLossBackward>)
tensor(106.8694, grad_fn=<NllLossBackward>)
tensor(47.1747, grad_fn=<NllLossBackward>)
tensor(35.4612, grad_fn=<NllLossBackward>)
tensor(28.1593, grad_fn=<NllLossBackward>)
tensor(34.4867, grad_fn=<NllLossBackward>)
tensor(31.3151, grad_fn=<NllLossBackward>)
Epoch-0 lr: 0.001 running_loss: 14.776168823242188
Epoch: [0][0/4]	Time 3.124 (3.124)	
tensor(32.8607, grad_fn=<NllLossBackward>)
Epoch: [1][0/9]	Time 7.350 (7.350)	Loss 32.8607 (32.8607)	
tensor(28.8626, grad_fn=<NllLossBackward>)
tensor(42.8860, grad_fn=<NllLossBackward>)
tensor(37.2594, grad_fn=<NllLossBackward>)
tensor(25.0614, grad_fn=<NllLossBackward>)
tensor(10.9538, grad_fn=<NllLossBackward>)
tensor(39.7883, grad_fn=<NllLossBackward>)
tensor(16.8993, grad_fn=<NllLossBackward>)
tensor(11.1821, grad_fn=<NllLossBackward>)
Epoch-1 lr: 0.001 run

tensor(1.3308, grad_fn=<NllLossBackward>)
tensor(0.1337, grad_fn=<NllLossBackward>)
Epoch-15 lr: 0.001 running_loss: 0.2624087929725647
Epoch: [15][0/4]	Time 3.039 (3.039)	
tensor(1.2514, grad_fn=<NllLossBackward>)
Epoch: [16][0/9]	Time 7.418 (7.418)	Loss 1.2514 (1.2514)	
tensor(1.3273, grad_fn=<NllLossBackward>)
tensor(1.3391, grad_fn=<NllLossBackward>)
tensor(1.2830, grad_fn=<NllLossBackward>)
tensor(1.3134, grad_fn=<NllLossBackward>)
tensor(1.3038, grad_fn=<NllLossBackward>)
tensor(1.3205, grad_fn=<NllLossBackward>)
tensor(1.2361, grad_fn=<NllLossBackward>)
tensor(0.1234, grad_fn=<NllLossBackward>)
Epoch-16 lr: 0.001 running_loss: 0.2441377192735672
Epoch: [16][0/4]	Time 3.073 (3.073)	
tensor(1.1855, grad_fn=<NllLossBackward>)
Epoch: [17][0/9]	Time 7.378 (7.378)	Loss 1.1855 (1.1855)	
tensor(1.2482, grad_fn=<NllLossBackward>)
tensor(1.2479, grad_fn=<NllLossBackward>)
tensor(1.1996, grad_fn=<NllLossBackward>)
tensor(1.2266, grad_fn=<NllLossBackward>)
tensor(1.2140, grad_fn=<NllLossBac